In [ ]:
import os

import time
import random
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from topology_generation import Topology
from adversaries import Adversaries
from objective_functions_cifar10 import DecentralizedCIFAR10
from resilient_algorithms import ResilientAlgorithms
from algorithmic_framework import DistributedAlgorithmFramework_ML

### CIFAR10 Experiments

In [ ]:
# variables for CIFAR-10 experiments
seed = 42   
alg_name = 'CWTM'  # chosen from ['DGD', 'CWTM', 'SDFD', 'R-SDMMFD']
n_adv = 1   

# fundamental parameters
n_nodes = 10
n_epochs = 3

# function parameters
batch_size = 64
bias_flag = False

# topology parameters
topo_type = 'r-robust'
period = 1
hyperparams_dict = {'threshold': 0.20, 'robustness': 4}
F = 1

# adversary model parameters
attack_info = {'type': 'perturbation', 'param': 0.01, 
               'perturbation_mode': 'gaussian', 'broadcast_flag': False}
adv_model = 'local'

# baseline parameters
baseline_epochs = 30
baseline_optimizer = 'SGD'
baseline_opt_config = {'learning_rate': 0.002, 'momentum': 0.9}

# framework parameters
state_inits = {'main': None, 'aux': None}
stepsize_init = {'type': 'constant', 'param': 0.04}
directory = f'../results/exp_cifar10_adv{n_adv}/'
eval_info = {'n_nodes': None, 'period': 5, 'n_eval_iters': 100, 'path_name': directory}

# miscellaneous parameters
self_flag = True
acceleration_flag = True
verify_flag = False
display_flag = False

In [ ]:
### create directory
if not os.path.exists(directory):
    os.makedirs(directory)

### set random seed
random.seed(seed)
np.random.seed(seed)
tf.experimental.numpy.random.seed(seed)


### initialize objects
topology = Topology(topo_type, n_nodes, period, **hyperparams_dict)
adversaries = Adversaries(topology.adjacencies[0], n_adv, adv_model, attack_info, display_flag)
function = DecentralizedCIFAR10(n_nodes, batch_size, bias_flag, adversaries.reg_indices, display_flag)
framework = DistributedAlgorithmFramework_ML(topology, function, adversaries, eval_info)
algorithm = ResilientAlgorithms(alg_name, F, self_flag, acceleration_flag, verify_flag, display_flag)

### set-up initialization and execute algorithm
start_time_main = time.time()
framework.initialization(n_epochs, state_inits, stepsize_init)
framework.distributed_algorithm(algorithm)
print(f"Time taken: {time.time() - start_time_main:.2f}s")

np.set_printoptions(precision=2)
print(f"Test Accuracy: {np.array(framework.metric_dict['test_acc_avg'])}")
print(f"Adversary Indices: {framework.adv_indices}")
print(f"Number of Edges: {int(np.sum(topology.adjacencies[0] / 2))}")

### train baseline
if alg_name == 'DGD':
    start_time_baseline = time.time()
    baseline_dict = function.train_baseline(baseline_epochs, baseline_optimizer, baseline_opt_config, directory)
    print(f"Baseline: Time {time.time() - start_time_baseline:.2f}s; Test Accuracy {baseline_dict['test_acc'][-1]} \n")

### CIFAR10 Temporary Plot

In [ ]:
x = framework.eval_timesteps
names = ['_acc', '_loss']
ylabels = ['Accuracy', 'Loss']
for name, ylabel in zip(names, ylabels):
    # plot average accuracies
    plt.plot(x, framework.metric_dict['train' + name + '_avg'], label='train', color='C0')
    plt.plot(x, framework.metric_dict['test' + name + '_avg'], label='test', color='C1')
    # plot worst accuracies
    plt.plot(x, framework.metric_dict['train' + name + '_w'], label='train (worst)', linestyle='dashed', color='C0')
    plt.plot(x, framework.metric_dict['test' + name + '_w'], label='test (worst)', linestyle='dashed', color='C1')
    plt.title(f"{alg_name} Algorithm; {attack_info['perturbation_mode']} {attack_info['param']}")
    plt.xlabel('Epoch')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid()
    plt.show()

### CIFAR10 Plot (attack strength)

In [ ]:
from plots import Plottings2


### set-up plotting parameters
# fundamental parameters
directory = '../results/!exp_cifar10/adv6_gaussian0050/'
attack_param = 0.050
n_plot_steps = None
y_upper_lim_acc = 85


# base plotting parameters
title_extension = r' ($\hat{\sigma}$ = ' + f'{attack_param})'
label_exts = ['', ' (worst)']
base_plt = {'ylog': False, 'label_exts': label_exts, 'title_ext': title_extension}

# metric groups
train_acc_group = ['train_acc_avg', 'train_acc_w']
bl_train_acc = 'train_acc_baseline'
train_acc_plt = {'nickname': 'train_accuracy', 'ylabel': 'Train Accuracy', 
                 'limit': [0, n_plot_steps, 0, y_upper_lim_acc], 'acc_flag': True}

test_acc_group = ['test_acc_avg', 'test_acc_w']
bl_test_acc = 'test_acc_baseline'
test_acc_plt = {'nickname': 'test_accuracy', 'ylabel': 'Test Accuracy',
                'limit': [0, n_plot_steps, 0, y_upper_lim_acc], 'acc_flag': True}

train_loss_group = ['train_loss_avg', 'train_loss_w']
bl_train_loss = 'train_loss_baseline'
train_loss_plt = {'nickname': 'train_loss', 'ylabel': 'Train Loss',
                    'limit': [0, n_plot_steps, None, None], 'acc_flag': False}

test_loss_group = ['test_loss_avg', 'test_loss_w']
bl_test_loss = 'test_loss_baseline'
test_loss_plt = {'nickname': 'test_loss', 'ylabel': 'Test Loss',
                    'limit': [0, n_plot_steps, None, None], 'acc_flag': False}


### create a list of dictionaries for each metric group
# create a list of dictionaries for each metric group
metric_groups = [{'metrics': train_acc_group, 'baseline': bl_train_acc, 'plt': train_acc_plt}, 
                 {'metrics': test_acc_group, 'baseline': bl_test_acc, 'plt': test_acc_plt},
                 {'metrics': train_loss_group, 'baseline': bl_train_loss, 'plt': train_loss_plt},
                 {'metrics': test_loss_group, 'baseline': bl_test_loss, 'plt': test_loss_plt}]

# include base_plt in each plt_dict
for group_dict in metric_groups:
    group_dict['plt'].update(base_plt)


### create Plottings object and plot results
plotting = Plottings2(directory)
plotting.plot_results(metric_groups)

### CIFAR10 Plot (adversarial agents)

In [ ]:
from plots import Plottings2


### set-up plotting parameters
# fundamental parameters
directory = '../results/!exp_cifar10/adv24_gaussian0035/'
n_adv = 24
n_plot_steps = None
y_upper_lim_acc = 85


# base plotting parameters
title_extension = r' ($| \mathcal{V}_{\mathcal{B}} |$ = ' + f'{n_adv})'
label_exts = ['', ' (worst)']
base_plt = {'ylog': False, 'label_exts': label_exts, 'title_ext': title_extension}

# metric groups
train_acc_group = ['train_acc_avg', 'train_acc_w']
bl_train_acc = 'train_acc_baseline'
train_acc_plt = {'nickname': 'train_accuracy', 'ylabel': 'Train Accuracy', 
                 'limit': [0, n_plot_steps, 0, y_upper_lim_acc], 'acc_flag': True}

test_acc_group = ['test_acc_avg', 'test_acc_w']
bl_test_acc = 'test_acc_baseline'
test_acc_plt = {'nickname': 'test_accuracy', 'ylabel': 'Test Accuracy',
                'limit': [0, n_plot_steps, 0, y_upper_lim_acc], 'acc_flag': True}

train_loss_group = ['train_loss_avg', 'train_loss_w']
bl_train_loss = 'train_loss_baseline'
train_loss_plt = {'nickname': 'train_loss', 'ylabel': 'Train Loss',
                    'limit': [0, n_plot_steps, None, None], 'acc_flag': False}

test_loss_group = ['test_loss_avg', 'test_loss_w']
bl_test_loss = 'test_loss_baseline'
test_loss_plt = {'nickname': 'test_loss', 'ylabel': 'Test Loss',
                    'limit': [0, n_plot_steps, None, None], 'acc_flag': False}


### create a list of dictionaries for each metric group
# create a list of dictionaries for each metric group
metric_groups = [{'metrics': train_acc_group, 'baseline': bl_train_acc, 'plt': train_acc_plt}, 
                 {'metrics': test_acc_group, 'baseline': bl_test_acc, 'plt': test_acc_plt},
                 {'metrics': train_loss_group, 'baseline': bl_train_loss, 'plt': train_loss_plt},
                 {'metrics': test_loss_group, 'baseline': bl_test_loss, 'plt': test_loss_plt}]

# include base_plt in each plt_dict
for group_dict in metric_groups:
    group_dict['plt'].update(base_plt)


### create Plottings object and plot results
plotting = Plottings2(directory)
plotting.plot_results(metric_groups)